In [63]:
#Imports 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

In [64]:
#Load items Data into a dictionary
items = {}
index = 0
infile = open('../input/aiml426-knapsack-data/100_995','r')
next(infile) #Skip First Line
for line in infile:
    instring = line.strip().split(' ')
    inints = (int(instring[0]),int(instring[1]))
    items[index] = inints
    index += 1

MAX_WEIGHT = 995
infile.close()
print(items)

In [65]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [66]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", random.randint, 0, 0)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [67]:
#Evaluation Function
def evalKnapsack(individual):
    value = 0.0
    index = 0
    
    for indicator in individual:
        if indicator == 1:
            itemtuple = items.get(index)
            value += itemtuple[0]
        index += 1
    
    return value, 

In [68]:
def feasible(individual):
    """Feasibility function for the individual. Returns True if feasible False
    otherwise."""
    weight = 0.0
    index = 0
    
    for indicator in individual:
        if indicator == 1:
            itemtuple = items.get(index)
            weight += itemtuple[1]
        index += 1
        
    if weight <= MAX_WEIGHT:
        return True
    return False

def distance(individual):
    """A distance function to the feasibility region."""
    return (individual[0] - 5.0)**2

In [69]:
#Register these above functions in DEAP toolbox
toolbox.register("evaluate", evalKnapsack)
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 0))
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb = 0.05)
toolbox.register("select", tools.selTournament, tournsize=5)

In [70]:
def main(seed):
    random.seed(seed)
    NGEN = 500
    MU = 500
    LAMBDA = 495
    CXPB = 0.8
    MUTPB = 0.2
    pop = toolbox.population(n=MU)
    
    hof = tools.HallOfFame(maxsize = 1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)

    return pop, stats, hof

In [71]:
results = []
for i in range(5):
    pop, stats, hof = main(i)
    results.append(evalKnapsack(hof[0])[0])
    print(results)

In [72]:
import statistics
print(results)
stnd_dev = statistics.stdev(results)
mean = statistics.mean(results)

print(stnd_dev)
print(mean)